<a href="https://colab.research.google.com/github/rrishabh145/ML-algorithms/blob/master/cat_VS_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART 1: Importing data from kaggle and setting up the notebook**

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [0]:
# Upload kaggle API key file
# Generate this file from Kaggle account
uploaded = files.upload()

In [0]:
# Make directory named kaggle and copy kaggle.json file there.
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/

# Change the permissions of the file.
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
# You can check if everything's okay by running this command.
! kaggle datasets list

In [0]:
# Downloading the dataset
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [0]:
# Unzipping the files into two folders
!unzip train.zip
!unzip test.zip 

**PART 2: Data processing**

In [0]:
import cv2  # to resize the image
import numpy as np 
import os   #to work with the directories
from random import shuffle  #to shuffle the data
from tqdm import tqdm   #for visualising looping

# Location of extracted data
TRAIN_DIR = '/content/train'
TEST_DIR = '/content/test'

IMG_SIZE = 50   # making all size 50x50
LR = 1e-3  # Learning Rate

# Saving a model name for future reference
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic-video')

In [0]:
# Function to manually one hot encode tha cat or dog label from image file names
def label_img(img):
  # dog.93.png will be split and 'dog' will be used as word label
  word_label = img.split('.')[-3]
  if word_label == 'cat': return[1,0]
  elif word_label == 'dog': return[0,1]

In [0]:
# function to process images to create training data
def create_train_data():
  training_data = []
  for img in tqdm(os.listdir(TRAIN_DIR)):
    label = label_img(img)  # getting the label of the image
    path = os.path.join(TRAIN_DIR, img) # getting the full parth of the image file
    
    # Converting images into grayscale and then resizing them into 50x50
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE) ) 
    training_data.append([np.array(img), np.array(label)]) #converting image to numpy array and saving

  shuffle(training_data)
  np.save('training_data.npy', training_data) #saving processed data to use in the model
  return training_data

In [0]:
# processing images to create testing_data
# prediction file of testing_data is to be returned
def process_test_data():
  testing_data = []
  for img in tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR, img) # getting the full parth of the image file
    img_num = img.split('.')[0]

    # Converting images into grayscale and then resizing them into 50x50
    img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
    testing_data.append([np.array(img), img_num]) #converting image to numpy array and saving

  np.save('test_data.npy',testing_data) #saving processed data to use in the model
  return testing_data

In [0]:
train_data = create_train_data() # This will create train data
# If you already have train data:
#train_data = np.load('training_data.npy')

**PART 3: Model initialisation, building and training**

In [0]:
# Setting the tensorflow version of colab notebook to tf 1.x
%tensorflow_version 1.x

# Installing tflearn
! pip install -q tflearn

In [0]:
# To rerun the code or training, uncomment the below code.
# This will reset the graph generated in the last training session

import tensorflow as tf
#tf.reset_default_graph()

In [0]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1], name = 'input')

# 6 layers of covnvnet followed by 1 layer of fully connected layers
convnet = conv_2d(convnet, 32, 2, activation = 'relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 64, 2, activation = 'relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 128, 2, activation = 'relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 128, 2, activation = 'relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 64, 2, activation = 'relu')
convnet = max_pool_2d(convnet,2)

convnet = conv_2d(convnet, 32, 2, activation = 'relu')
convnet = max_pool_2d(convnet,2)

convnet = fully_connected(convnet, 1024, activation = 'relu')
convnet = dropout(convnet,0.8)

convnet = fully_connected(convnet, 2, activation = 'softmax')
convnet = regression(convnet, optimizer = 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')

model =tflearn.DNN(convnet, tensorboard_dir = 'log')

In [0]:
# if a partially trained model is available in the direcetory, loading that checkpoint file to continue training
# the model further
if os.path.exists('{},meta'.format(MODEL_NAME)):
  model.load(MODEL_NAME)
  print('model loaded!')

In [0]:
# seperating train and test data
train = train_data[:-500]
test = train_data[-500:]

# seperating the features and labels for TRAIN data
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

# seperating the features and labels for TEST data
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [0]:
# Training the model and validating on holdout set (test set)
model.fit({'input':X}, {'targets':Y}, n_epoch = 12, validation_set = ({'input':test_x},{'targets':test_y}), snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)

In [0]:
# saving model for future predictions or retraining the model
model.save(MODEL_NAME)

In [0]:
# visualise contents of the model and training on tensorboard
#%tensorboard --logdir logs

**PART 4: Predictions and visualisations on unlabeled data using trained model**

In [0]:
import matplotlib.pyplot as plt

# if you dont already have this file yet
test_data = process_test_data()
# if you already have it
#test_data = np.load('test_data.npy')

fig = plt.figure()

# run first 12 test images though the model and plot on a figure along with
# their classification labels to check if the model is working properly and accurately
for num, data in enumerate(test_data[:12]):
  # cat : [1,0]
  # dog : [0,1]
  
  img_num = data[1] #image label number
  img_data = data[0]  #image in the form of numpy array

  y = fig.add_subplot(3,4,num+1) 
  orig = img_data
  data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)

  model_out = model.predict([data])[0]  # predicting on test images
  
  # setting label to be displayed in the graph
  if np.argmax(model_out) == 1: str_label = 'Dog'
  else: str_label = 'Cat'

  y.imshow(orig, cmap = 'gray') #displaying image in grayscale
  plt.title(str_label)
  y.axes.get_xaxis().set_visible(False)
  y.axes.get_yaxis().set_visible(False)

plt.show()

**PART 5: Creating a kaggle submission file**

In [0]:
# creating a new empty csv file
with open('submission-file.csv','w') as f:
  f.write('id,label\n') #It will over-write the whole file with the above data

In [0]:
# writing the predictions in the created csv file in append mode
with open('submission-file.csv','a') as f:
  for data in tqdm(test_data):
    img_num = data[1]  #image label number
    img_data = data[0]  #image in the form of numpy array
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]  # predicting on test images
    # model_out will be in the form :
    # [ 0.25, 0.75 ]
    f.write('{},{}\n'.format(img_num, model_out[1]))

In [0]:
!kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f submission-file.csv -m "My first Kaggle Submission"